### Structural Similarity Index (SSIM)
The SSIM extracts 3 key features from an image:
- Luminance
- Contrast
- Structure

The comparison between the two images is performed on the basis of these 3 features.

c1 and c2 below are constants to stabilise the division with a weak denominator.<br>
They are usually derived from (k1 x L)^2 and (k2 x L)^2, where L is the dynamic range of the pixel values <br>
(Ex: 255 for 8 bit images)

In [1]:
import cv2
import numpy as np

In [2]:
# Constants for luminance and contrast
c1 = 6.5025
c2 = 58.5225

In [3]:
img1 = cv2.imread('images/messi.png', 0)

In [4]:
# Creating a noisy image from img1
mean = 0
std = 25
noise = np.random.normal(mean, std, img1.shape).astype(np.uint8)
noisy_image = cv2.add(img1, noise)
img2 = noisy_image

In [5]:
# Converting to float for squaring
img1 = np.float32(img1)
img2 = np.float32(img2)
img1_sq = img1 * img1
img2_sq = img2 * img2
img1_img2 = img1 * img2

Now we calculate <b>the luminance, μ (Mu)</b>. Luminance is measured by averaging over all the pixel values. But we don't apply the metric globally. It's better to apply the metrics regionally and take the overall mean. So we compute the local means μ1 and μ2 using gaussian blur.

In [7]:
# Luminance
# applying GaussianBlur with (11,11) kernel where mean=st_dev=1.5
mu1 = cv2.GaussianBlur(img1, (11, 11), 1.5)
mu2 = cv2.GaussianBlur(img2, (11, 11), 1.5)

# Taking squares and product of the means.
mu1_sq = mu1 * mu1
mu2_sq = mu2 * mu2
mu1_mu2 = mu1 * mu2

<b>Contrast</b> is measured by taking the standard deviation (sq. root of variance) of all pixel values.
It is denoted by <b>σ (sigma)</b>.<br>

In [8]:
# Contrast
sigma1_sq = cv2.GaussianBlur(img1_sq, (11, 11), 1.5)
sigma1_sq -= mu1_sq
sigma2_sq = cv2.GaussianBlur(img2_sq, (11, 11), 1.5)
sigma2_sq -= mu2_sq

# sigma12 is the covariance and represents the structure correlation between the images.
sigma12 = cv2.GaussianBlur(img1_img2, (11, 11), 1.5)
sigma12 -= mu1_mu2


In [10]:
# Luminance term correlating the two images.
numerator1 = 2 * mu1_mu2 + c1
denominator1 = mu1_sq + mu2_sq + c2 

# Contrast and structure term correlating the two images.
numerator2 = 2 * sigma12 + c2
denominator2 = sigma1_sq + sigma2_sq + c2

In [11]:
# Combined SSIM Index
ssim_score = (numerator1 * numerator2) / (denominator1 * denominator2)

This system calculates the Structural Similarity Index between 2 given images which is a value between -1 and +1. A value of +1 indicates that the 2 given images are very similar or the same while a value of -1 indicates the 2 given images are very different. Often these values are adjusted to be in the range [0, 1], where the extremes hold the same meaning.

In [12]:
print(ssim_score.mean())

0.10644442
